# DocAI Splitting Overlapping Entities


* Author: docai-incubator@google.com


## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the DocAI Incubator Team. No guarantees of performance are implied.

## Purpose and Description
This tool uses exported labeled json to separate a pair of entities that are overlapped due to labeling into two individual entities. 

## Prerequisites

1. Vertex AI Notebook
2. Parsed json files in GCS Folder.
3. Output folder to upload the updated json files.

## Step by Step procedure 

### 1. Input details


In [3]:
# input details
# INPUT : storage bucket name
input_path = "gs://xxxxx/xxxxxxxx"
# OUTPUT : storage bucket's path
output_path = "gs://xxxxxx/xxxxxxxx"

list_of_pair_of_entities = [
    ("currency", "invoice_id"),
    ("purchase_order", "delivery_date"),
]  # List of pair of entities that needs to be splitted.
# Also, the entity name should be mentioned like this (small_entity,large_entity)

input_path : GCS Path for input json files<br>
output_path: GCS Path for output json files<br>
list_of_pair_of_entities: [('customer_account_name','ship_to_address')]

<div><i><b>Note:</b> List of pairs of entities that need to be splitted. Also, the entity name should be mentioned like this (small_entity,large_entity)</i><div>

### 2. Output

The output json after execution of the code have individual entities.
<img src="./Images/overlapping_split_output_1.png" width=800 height=400 alt="Overlapping entity split output">

### 3. Run the code

In [ ]:
%pip install tqdm
%pip install google.cloud

In [ ]:
# Run this cell to download utilities module
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/document-ai-samples/main/incubator-tools/best-practices/utilities/utilities.py

In [ ]:
from io import BytesIO
import json, copy
from google.cloud import storage
from tqdm.notebook import tqdm
from utilities import (
    file_names,
    documentai_json_proto_downloader,
    bb_intersection_over_union,
    store_document_as_json,
)
from google.cloud import documentai_v1beta3 as documentai
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union


input_storage_bucket_name = input_path.split("/")[2]
input_bucket_path_prefix = "/".join(input_path.split("/")[3:])
output_storage_bucket_name = output_path.split("/")[2]
output_bucket_path_prefix = "/".join(output_path.split("/")[3:])

json_files = file_names(input_path)[1].values()
list_of_files = [i for i in list(json_files) if i.endswith(".json")]


def get_coordinate(entity: documentai.Document.Entity, arg: str) -> int:
    """
    To get the coordinates according to the entity coordinate name

    Parameters
    ----------
    entity : documentai.Document.Entity
        The entity object from the input document
    arg : str
        The coordinate name of the entity bounding box

    Returns
    -------
    int
        Returns the bounding box coordinate according to the name of the coordinates.
    """
    x_list = []
    y_list = []
    for i in entity.page_anchor.page_refs:
        for j in i.bounding_poly.normalized_vertices:
            x_list.append(j.x)
            y_list.append(j.y)

    if arg == "x_min":
        return min(x_list)
    if arg == "y_min":
        return min(y_list)
    if arg == "x_max":
        return max(x_list)
    if arg == "y_max":
        return max(y_list)


def get_entity_coordinates(
    entity1: documentai.Document.Entity, entity2: documentai.Document.Entity
) -> Tuple[float, float]:
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    entity1 : documentai.Document.Entity
            The first entity from the elements of list_of_pair_of_entities
    entity2 : documentai.Document.Entity
            The second entity from the elements of list_of_pair_of_entities

    Returns
    -------
    List
        entity_1_coordinates : coordinates of first entity
        entity_2_coordinates : coordinates of second entity
    """
    entity_1_coordinates = [
        get_coordinate(entity1, "x_max"),
        get_coordinate(entity1, "x_min"),
        get_coordinate(entity1, "y_max"),
        get_coordinate(entity1, "y_min"),
    ]
    entity_2_coordinates = [
        get_coordinate(entity2, "x_max"),
        get_coordinate(entity2, "x_min"),
        get_coordinate(entity2, "y_max"),
        get_coordinate(entity2, "y_min"),
    ]

    return entity_1_coordinates, entity_2_coordinates


def get_token(token: documentai.Document.PageAnchor.PageRef, param: str) -> int:
    """
    To get the coordinates of the token by the coordinate name.

    Parameters
    ----------
    token : documentai.Document.PageAnchor.PageRef
            The token object from the entity.
    param : str
            The coordinates name to fetch the value.

    Returns
    -------
    int :
        Coordinates of thr token according to the param(parameter).
    """
    x_list = []
    y_list = []
    for j in token.layout.bounding_poly.normalized_vertices:
        x_list.append(j["x"])
        y_list.append(j["y"])
    if param == "x_min":
        return min(x_list)
    if param == "x_max":
        return max(x_list)
    if param == "y_min":
        return min(y_list)
    if param == "y_max":
        return max(y_list)


def find_textSegment_list(
    x_min: int, y_min: int, x_max: int, y_max: int, js: documentai.Document, page: int
) -> List[documentai.Document.TextAnchor.TextSegment]:
    """
    To get the text segment list.

    Parameters
    ----------
    x_min : The minimum value of x coordinate (top left).
    y_min : The minimum value of y coordinate (bottom left).
    x_max : The maximum value of x coordinate (top right).
    y_max : The maximum value of y coordinate (bottom right).
    js    : documentai.Document
            The Document proto object from the entities.
    page : int
            The page number.

    Returns
    -------
    List[documentai.Document.TextAnchor.TextSegment] :
        List of text segment.
    """
    textSegments_list = []
    for token in js.pages[page].tokens:
        token_xMin = get_token(token, "x_min")
        token_xMax = get_token(token, "x_max")
        token_yMin = get_token(token, "y_min")
        token_yMax = get_token(token, "y_max")
        if (
            token_xMin >= x_min
            and token_xMax <= x_max
            and token_yMin >= y_min
            and token_yMax <= y_max
        ):
            textSegments_list.extend(token.layout.text_anchor.text_segments)

    return textSegments_list


def split_overlapping_entities(
    large_entity: documentai.Document.Entity,
    small_entity: documentai.Document.Entity,
    js: documentai.Document,
    page: str,
) -> documentai.Document.Entity:
    """It will append new entities to Document Proto by replacing the text of small entitiy from large entity and also the coordinates

    Args:
        doc (documentai.Document): It is Document proto object
        total_amount_type (str): Its value is set as type for an entity, here for all properties in an entity
        list_total_amount (List[str]): It is a list of header words which will be used to identity and the values under those headers will be tagged with child type `total_amount_type`

    Returns:
        documentai.Document: It is Document proto object, which contains newly added entities as well
    """

    new_entity = {}
    new_entity["type"] = large_entity.type
    new_entity["mention_text"] = large_entity.mention_text.replace(
        small_entity.mention_text, ""
    )
    text_anchor = {}
    text_anchor["content"] = large_entity.mention_text.replace(
        small_entity.mention_text, ""
    )
    A = {
        "x": get_coordinate(small_entity, "x_min"),
        "y": get_coordinate(small_entity, "y_max"),
    }
    B = {
        "x": get_coordinate(small_entity, "x_max"),
        "y": get_coordinate(small_entity, "y_max"),
    }
    C = {
        "x": get_coordinate(large_entity, "x_max"),
        "y": get_coordinate(large_entity, "y_max"),
    }
    D = {
        "x": get_coordinate(large_entity, "x_min"),
        "y": get_coordinate(large_entity, "y_max"),
    }
    new_entity["page_anchor"] = large_entity.page_anchor
    new_entity["page_anchor"]["page_refs"][0]["bounding_poly"][
        "normalized_vertices"
    ] = [A, B, C, D]
    new_entity["page_anchor"]["page_refs"][0]["page"] = str(page)
    text_anchor["text_segments"] = find_textSegment_list(
        A["x"] - 0.005, A["y"] - 0.005, C["x"] + 0.005, C["y"] + 0.005, js, page
    )
    new_entity["text_anchor"] = text_anchor
    print()
    return new_entity


for k in tqdm(range(0, len(list_of_files))):
    new_entities = []
    print("\nProcessing >>> ", list_of_files[k])
    document = documentai_json_proto_downloader(
        input_storage_bucket_name, list_of_files[k]
    )
    try:
        for j in list_of_pair_of_entities:
            print(j)
            small_entity = j[0]
            large_entity = j[1]
            list_of_small_entity = []
            for entity in document.entities:
                if entity.type == small_entity:  # or entity['type']=="ship_to_address"
                    list_of_small_entity.append(entity)
            for i in list_of_small_entity:
                page_i = 0
                if i.page_anchor.page_refs[0].page:
                    page_i = int(i.page_anchor.page_refs[0].page)
                for entity in document.entities:
                    if entity.type == large_entity:
                        page = 0
                        if entity.page_anchor.page_refs[0].page:
                            page = int(entity.page_anchor.page_refs[0].page)
                        if page == page_i:
                            (
                                entity1_coordinate,
                                entity2_coordinate,
                            ) = get_entity_coordinates(entity, i)
                            iou = bb_intersection_over_union(
                                entity1_coordinate, entity2_coordinate
                            )
                            if iou > 0.0:
                                new_entity = split_overlapping_entities(
                                    entity, i, document, page
                                )
                                new_entities.append(new_entity)
                                document.entities.remove(entity)
                                document.entities.append(new_entity)
    except Exception as e:
        print(
            ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>"
            + list_of_files[k]
            + " was not processed successfully!!!"
        )
        print(e)
        continue
    store_document_as_json(
        documentai.Document.to_json(document),
        output_storage_bucket_name,
        output_bucket_path_prefix + "/" + list_of_files[k].split("/")[-1],
    )